# Heat Index Applications
This notebook walks through the [NOAA Heat Index](https://www.weather.gov/ama/heatindex) using climate data projections in the Analytics Engine. 

To execute a given 'cell' of this notebook, place the cursor in the cell and press the 'play' icon, or simply press shift+enter together. Some cells will take longer to run, and you will see a [$\ast$] to the left of the cell while AE is still working.

**Intended Application**: As a user, I want to **<span style="color:#FF0000">understand trends in Heat Index across my region</span>** by:
1. Calculating the number of hours per day throughout the year of high Heat Index values
2. Understanding the trend in nighttime temperatures that are above an 80°F Heat Index
3. Determining the historical and projected number of days with a high Heat Index per month

**Runtime**: With the default settings, this notebook takes approximately **16 minutes** to run from start to finish. Modifications to selections may increase the runtime. 

## Step 0: Set-up

First, we'll import the python library [climakitae](https://github.com/cal-adapt/climakitae), our AE toolkit for climate data analysis, along with this specific functions from that library that we'll use in this notebook, as well as any other necessary python libraries to aid in analysis.

In [1]:
import climakitae as ck
import climakitaegui as ckg
import pandas as pd
import panel as pn
pn.extension()

from climakitae.util.utils import (
    read_csv_file, get_closest_gridcell, compute_multimodel_stats, 
    trendline, summary_table, convert_to_local_time
)

from climakitae.core.data_interface import get_data
from climakitae.core.data_interface import DataInterface

## Step 1: Select data

### 1a) Grab location of interest by latitude and longitude
First we'll grab specific locations of interest, by using the latitude and longitude of a weather station, and provide code to input a custom lat-lon location. Furthermore, we will **not** be retrieving the actual station data that is bias-corrected to that station for this example. At present bias-corrected station data on the Analytics Engine only provides air temperature as a variable, and for Heat Index we must also have either dew point temperature (coming soon!) or relative humidity. So for the time being, we will retrieve **non-bias corrected** data at the location of interest.

Note: For demonstration purposes we are selecting only ten years of data. We recomend using a longer period (at least 30 years) for conducting a scientific analysis.

In [11]:
# select historical data
heatidx_hist_hour = get_data(
    variable="NOAA Heat Index",
    resolution = "9 km",
    timescale = "hourly",
    data_type = "Gridded",
    area_subset = "CA Electric Load Serving Entities (IOU & POU)",
    cached_area = ["Pacific Gas & Electric Company"],
    time_slice = (1981, 1990) # Short period for quick demonstration
)

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!! Returned data array is huge. Operations could take 10x to infinity longer than 1GB of data !!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

WARNING
-------
You have retrieved data for more than one SSP, but not all ensemble members for each GCM are available for all SSPs.

As a result, some scenario and simulation combinations may contain NaN values.

If you want to remove these empty simulations, it is recommended to first subset the data object by each individual scenario and then dropping NaN values.


In [12]:
#'location_subset': ['entire domain']
#'location_subset': ['coordinate selection']

We will first look at the Fresno Airport weather station.

In [13]:
from climakitae.core.paths import stations_csv_path
wx_stns = read_csv_file(stations_csv_path, index_col=[0])
wx_stns.head(5)

,state,station,city,ID,LAT_Y,LON_X,station id,elevation
0,CA,Bakersfield Meadows Field (KBFL),Bakersfield,KBFL,35.43424,-119.05524,72384023155,149.3
1,CA,Blythe Asos (KBLH),Blythe,KBLH,33.61876,-114.71451,74718823158,120.4
2,CA,Burbank-Glendale-Pasadena Airport (KBUR),Burbank,KBUR,34.19966,-118.36543,72288023152,222.7
3,CA,Needles Airport (KEED),Needles,KEED,34.76783,-114.61842,72380523179,270.6
4,CA,Fresno Yosemite International Airport (KFAT),Fresno,KFAT,36.77999,-119.72016,72389093193,101.9


In [14]:
station_name = 'Fresno Yosemite International Airport (KFAT)'
one_stn = wx_stns.loc[wx_stns['station'] == station_name]

stn_lat = one_stn.LAT_Y.values[0]
stn_lon = one_stn.LON_X.values[0]
print(stn_lat, stn_lon)

36.77999 -119.72016


If you would like to provide your own latitude and longitude coordinates, you can also customize the cell below and pass your own values. However, if your location is outside of the default cached area (for example, we're looking at the PG&E service territory), you'll also need to reset the `selections.cached_area` to one that is more appropriate. You can check which options are available in the "Subset the data by" and "Location selection" dropdown menus in `selections.show()`.

In [15]:
# stn_lat = YOUR_LAT_HERE
# stn_lon = YOUR_LON_HERE
# selections.cached_area = ["YOUR CHOICE HERE"] # if different cached area from default

Becasuse the dynamically downscaled WRF data in the Cal-Adapt: Analytics Engine is in UTC time, we'll convert to the timezome of the station we've selected. This is particularly important for determining the timing of the daily maximum and minimum temperatures. For a station located in Pacific Time (US), UTC time places the daily minimum "in" the day prior because UTC is 8 hours ahead of Pacific! The handy `convert_to_local_time` function corrects for this, and ensures that the resulting high and low temperatures are within the same daily timestamp. 

In [16]:
heatidx_hist_hour = convert_to_local_time(heatidx_hist_hour, (1981,1990), DataInterface()._geographies, "CA Electric Load Serving Entities (IOU & POU)")

Data converted to America/Los_Angeles timezone.


Now, we'll use the latitude and longitude values to retrieve the historical data at that gridcell. 

In [17]:
heatidx_hist_hour = get_closest_gridcell(heatidx_hist_hour, stn_lat, stn_lon)

Input coordinates: (36.78, -119.72)
Nearest grid cell coordinates: (36.78, -119.73)


Last, we load the data into memory. Because we are loading and computing on the entire dataset, this next code cell will take about **8 minutes**, hang tight! You will also see two warnings - don't worry! These are because of "no data" outside the spatial extent selected. 

In [18]:
heatidx_hist_hour = ck.load(heatidx_hist_hour)

Processing data to read 8.02 MB of data into memory... Complete!


### 1b) Retrieve the projected Heat Index data
Next we will repeat the same data retrieval as we did for the historical data so we can compare. Like the historical data, retrieving the hourly projections data will also take a few minutes - hang tight!

In [8]:
# select future data
# select historical data
heatidx_proj_hour = get_data(
    variable="NOAA Heat Index",
    resolution = "9 km",
    timescale = "hourly",
    data_type = "Gridded",
    scenario = ["SSP 3-7.0"],
    area_subset = "CA Electric Load Serving Entities (IOU & POU)",
    cached_area = ["Pacific Gas & Electric Company"],
    time_slice = (2040, 2049) # Short period for quick demonstration
)

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!! Returned data array is huge. Operations could take 10x to infinity longer than 1GB of data !!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



In [9]:
heatidx_proj_hour = convert_to_local_time(heatidx_proj_hour, (1981,1990), DataInterface()._geographies, "CA Electric Load Serving Entities (IOU & POU)")
heatidx_proj_hour = get_closest_gridcell(heatidx_proj_hour, stn_lat, stn_lon)

Data converted to America/Los_Angeles timezone.
Input coordinates: (36.78, -119.72)
Nearest grid cell coordinates: (36.78, -119.73)


Now, we will load in the hourly heat index projections. Because we are loading and computing on the entire original dataset, this will take a few minutes. Hang tight! You'll also see the same data warnings again.

In [10]:
heatidx_proj_hour = ck.load(heatidx_proj_hour)

Processing data to read 2.68 MB of data into memory... Complete!


## Step 2: Calculate the number of hours throughout the year above a threshold

### 2a) Sum the hours per day

Let's next determine what part of the day is above a Heat Index threshold, as well as how many hours in each day are above our designated threshold. The NOAA Heat Index  "kicks in" once the [NOAA Heat Index](https://www.noaa.gov/sites/default/files/2022-05/heatindex_chart_rh.pdf) value is above 80°F: prolonged exposure to a heat index above 80°F becomes dangerous to many people and especially vulnerable communities. The Occupational Safety and Health Administration (OSHA) uses the Heat Index to determine the risk of heat-related illness and protections for outdoor workers. Below are the [specific thresholds used by OSHA](https://www.nalc.org/workplace-issues/body/OSHA-Using-the-Heat-Index-A-Guide-for-Employers.pdf). We note that these values are slightly different than those used by the [NOAA Heat Index](https://www.noaa.gov/sites/default/files/2022-05/heatindex_chart_rh.pdf) system. We **strongly recommend** looking at multiple thresholds to understand Heat Index trends. 

| Classification | Heat Index |
|----------------|------------|
| Caution | <91°F |
| Moderate | 91 - 103°F |
| High | 103 - 115°F |
| Very High to Extreme | 115+°F |

We'll start with 80°F as our default threshold.

In [19]:
hi_threshold = 80

In [20]:
# counts the number of hours in each day above the heat index threshold
num_heatidx_histhours = (heatidx_hist_hour >= hi_threshold).resample(time="1D").sum()
num_heatidx_histhours.name = "Hours per day above Heat Index threshold of {}°F".format(
    hi_threshold
)

Let's identify one year to visualize the cycle in Heat Index temperatures. We are selecting 2000 here, but the commented out line of code below illustrates how to look at all of the data here. Just comment out the first line under visualize by adding a `#` symbol, and uncommenting the line below by removing the `#` symbol.

In [22]:
data_one_year = num_heatidx_histhours.sel(time="1985")

# visualize
data_one_year.hvplot.line(x="time", by="simulation")  # a specific year
# num_heatidx_histhours.hvplot.line(x='time', by='simulation') # all years

:DynamicMap   [scenario]
   :NdOverlay   [simulation]
      :Curve   [time]   (Hours per day above Heat Index threshold of 80°F)

Heat Index values above our selected threshold (default is 80°F) begin to pick up in March for Fresno, and tail off in November. While high Heat Index values are critical to know during the summer months, we should be aware of high Heat Index values throughout the rest of the year as well. 

### 2b) Sum the total hours per year
It may also be useful to know how many hours in the entire year are above the threshold in order to see the trends over time. We'll calculate this next.

In [23]:
# sum per year
num_heatidx_hist_hours_per_year = num_heatidx_histhours.groupby("time.year").sum("time")
num_heatidx_hist_hours_per_year.name = (
    "Hours per year above Heat Index threshold of {}°F".format(hi_threshold)
)

In [24]:
# visualize
num_heatidx_hist_hours_per_year.hvplot.line(x="year", by="simulation")

:DynamicMap   [scenario]
   :NdOverlay   [simulation]
      :Curve   [year]   (Hours per year above Heat Index threshold of 80°F)

For context, there are 8760 hours in a year. Let's also look at the future data to understand the projected trends in the number of high Heat Index hours per year. 

In the following cells, we'll do all of the computation above in a single go, to condense the number of cells to run. You don't need to modify anything, unless you made changes above. 

In [25]:
# counts the number of hours in each day above the heat index threshold
num_heatidx_projhours = (heatidx_proj_hour >= hi_threshold).resample(time="1D").sum()
num_heatidx_projhours.name = "Hours per day above Heat Index threshold of {}°F".format(
    hi_threshold
)

# sum per year
num_heatidx_proj_hours_per_year = num_heatidx_projhours.groupby("time.year").sum("time")
num_heatidx_proj_hours_per_year.name = (
    "Hours per year above Heat Index threshold of {}°F".format(hi_threshold)
)

In [26]:
# visualize
num_heatidx_proj_hours_per_year.hvplot.line(x="year", by="simulation")

:DynamicMap   [scenario]
   :NdOverlay   [simulation]
      :Curve   [year]   (Hours per year above Heat Index threshold of 80°F)

### 2c) Export counts of hours per year
First, we'll calculate summary statistics for our data object with a handy function `compute_multimodel_stats`, which will provide the min, max, mean, and median of our data. We'll also use a helper function `summary_table` which will transform our data objects into an easy-to-read dataframe. This dataframe will be exported as a .csv file and will be available in the file tree on the lefthandside of your screen. You can double click on it to view in a new window, or right-click to download. 

In [27]:
# calculate statistics
num_heatidx_hist_hours_per_year = compute_multimodel_stats(
    num_heatidx_hist_hours_per_year
)

# table format with summary stats
df_to_export = summary_table(num_heatidx_hist_hours_per_year)
df_to_export.head(5)  # see first 5 rows

Hours per year above Heat Index threshold of 80°F  \
simulation                               WRF_CESM2_r11i1p1f1   
scenario                              Historical + SSP 3-7.0   
year                                                           
1980                                                     0.0   
1981                                                  1349.0   
1982                                                  1264.0   
1983                                                  1227.0   
1984                                                  1290.0   

                                                          \
simulation                                                 
scenario   Historical + SSP 2-4.5 Historical + SSP 5-8.5   
year                                                       
1980                          0.0                    0.0   
1981                       1349.0                 1349.0   
1982                       1264.0                 1264.0   
1983                       1227.0                 1227.0   
1984                       1290.0                 1290.0   

                                                            \
simulation WRF_CNRM-ESM2-1_r1i1p1f2                          
scenario     Historical + SSP 3-7.0 Historical + SSP 2-4.5   
year                                                         
1980                            0.0                    0.0   
1981                         1187.0                    0.0   
1982                         1150.0                    0.0   
1983                         1260.0                    0.0   
1984                          993.0                    0.0   

                                                              \
simulation                        WRF_EC-Earth3-Veg_r1i1p1f1   
scenario   Historical + SSP 5-8.5     Historical + SSP 3-7.0   
year                                                           
1980                          0.0                        0.0   
1981                          0.0                     1283.0   
1982                          0.0                     1688.0   
1983                          0.0                     1518.0   
1984                          0.0                     1473.0   

                                                          \
simulation                                                 
scenario   Historical + SSP 2-4.5 Historical + SSP 5-8.5   
year                                                       
1980                          0.0                    0.0   
1981                          0.0                    0.0   
1982                          0.0                    0.0   
1983                          0.0                    0.0   
1984                          0.0                    0.0   

                                   ...                         \
simulation WRF_EC-Earth3_r1i1p1f1  ...        simulation mean   
scenario   Historical + SSP 3-7.0  ... Historical + SSP 5-8.5   
year                               ...                          
1980                          0.0  ...                  0.000   
1981                       1581.0  ...                168.625   
1982                       1497.0  ...                158.000   
1983                       1414.0  ...                153.375   
1984                       1637.0  ...                161.250   

                                                          \
simulation         simulation min                          
scenario   Historical + SSP 3-7.0 Historical + SSP 2-4.5   
year                                                       
1980                          0.0                    0.0   
1981                       1187.0                    0.0   
1982                       1027.0                    0.0   
1983                        853.0                    0.0   
1984                        993.0                    0.0   

                                                          \
simulation                   

In [28]:
fn = "num_heatidx_hours_per_year_{}".format(
    station_name.replace(" ", "_").replace("(", "").replace(")", "")
)
df_to_export.to_csv("{}.csv".format(fn))

### 2d) Calculate the number of Heat Index days per month
Alternatively, let's determine the number of days per month with Heat Index values above the threshold, as this may be useful for seasonal information. From the hourly Heat Index data, we'll now calculate the daily maximum heat index value. It is important to note that we will calculate the daily max heat index from the hourly heat index data, rather than taking the daily max air temperature and the average relative humidity, which artificially inflates the daily heat index value. 

If the daily median heat index is more relevant to your needs, we also provide the option in the cell below to calculate this instead. 

In [29]:
# historical
heatidx_hist_day = heatidx_hist_hour.resample(time="1D").max()  # daily max
# heatidx_hist_day = heatidx_hist_hour.resample(time='1D').median() # daily median

# future
heatidx_proj_day = heatidx_proj_hour.resample(time="1D").max()  # daily max
# heatidx_proj_day = heatidx_proj_hour.resample(time='1D').median() # daily median

We'll also reset our Heat Index threshold to be 91°F to assess a higher threshold. 

In [30]:
hi_threshold = 91

In [31]:
num_heatidx_histmonths = (heatidx_hist_day >= hi_threshold).resample(time="1M").sum()
num_heatidx_histmonths.name = (
    "Days per month above Heat Index threshold of {}°F".format(hi_threshold)
)

/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/groupers.py:487: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


In [32]:
num_heatidx_histmonths.hvplot.line(x="time", by="simulation")

:DynamicMap   [scenario]
   :NdOverlay   [simulation]
      :Curve   [time]   (Days per month above Heat Index threshold of 91°F)

### 2e) Export the counts of days per month of high Heat Index values

Like steps 2c, we'll calculate the min, max, mean, and median trends and format a table for easy export. 

In [33]:
# calculate summary stats
num_heatidx_histmonths_stats = compute_multimodel_stats(num_heatidx_histmonths)

# table format with summary stats
df_to_export = summary_table(num_heatidx_histmonths_stats)
df_to_export.head(5)  # see first 5 rows

Days per month above Heat Index threshold of 91°F  \
simulation                               WRF_CESM2_r11i1p1f1   
scenario                              Historical + SSP 3-7.0   
time                                                           
1980-12-31                                               0.0   
1981-01-31                                               0.0   
1981-02-28                                               0.0   
1981-03-31                                               0.0   
1981-04-30                                               0.0   

                                                          \
simulation                                                 
scenario   Historical + SSP 2-4.5 Historical + SSP 5-8.5   
time                                                       
1980-12-31                    0.0                    0.0   
1981-01-31                    0.0                    0.0   
1981-02-28                    0.0                    0.0   
1981-03-31                    0.0                    0.0   
1981-04-30                    0.0                    0.0   

                                                            \
simulation WRF_CNRM-ESM2-1_r1i1p1f2                          
scenario     Historical + SSP 3-7.0 Historical + SSP 2-4.5   
time                                                         
1980-12-31                      0.0                    0.0   
1981-01-31                      0.0                    0.0   
1981-02-28                      0.0                    0.0   
1981-03-31                      0.0                    0.0   
1981-04-30                      0.0                    0.0   

                                                              \
simulation                        WRF_EC-Earth3-Veg_r1i1p1f1   
scenario   Historical + SSP 5-8.5     Historical + SSP 3-7.0   
time                                                           
1980-12-31                    0.0                        0.0   
1981-01-31                    0.0                        0.0   
1981-02-28                    0.0                        0.0   
1981-03-31                    0.0                        0.0   
1981-04-30                    0.0                        0.0   

                                                          \
simulation                                                 
scenario   Historical + SSP 2-4.5 Historical + SSP 5-8.5   
time                                                       
1980-12-31                    0.0                    0.0   
1981-01-31                    0.0                    0.0   
1981-02-28                    0.0                    0.0   
1981-03-31                    0.0                    0.0   
1981-04-30                    0.0                    0.0   

                                   ...                         \
simulation WRF_EC-Earth3_r1i1p1f1  ...        simulation mean   
scenario   Historical + SSP 3-7.0  ... Historical + SSP 5-8.5   
time                               ...                          
1980-12-31                    0.0  ...                    0.0   
1981-01-31                    0.0  ...                    0.0   
1981-02-28                    0.0  ...                    0.0   
1981-03-31                    0.0  ...                    0.0   
1981-04-30                    0.0  ...                    0.0   

                                                          \
simulation         simulation min                          
scenario   Historical + SSP 3-7.0 Historical + SSP 2-4.5   
time                                                       
1980-12-31                    0.0                    0.0   
1981-01-31                    0.0                    0.0   
1981-02-28                    0.0                    0.0   
1981-03-31                    0.0                    0.0   
1981-04-30                    0.0                    0.0   

                                                          \
simulation                   

In [34]:
fn = "num_heatidx_histmonths_{}".format(
    station_name.replace(" ", "_").replace("(", "").replace(")", "")
)
df_to_export.to_csv("{}.csv".format(fn))

## Step 3: Understand trends in nighttime temperatures
Next we'll look specifically at nighttime temperatures in order to assess when it may be too hot outside for worker safety and for assets to cool down. 

### 3a) Subset for nighttime hours
First, let's subset our hourly Heat Index data specifically for the nighttime hours. We will use 8pm-6am as "nighttime", but you can modify based on your needs as well.

In [35]:
night_subset = [20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6]  # 24 hour time
heatidx_hist_nighthours = heatidx_hist_hour.isel(
    time=heatidx_hist_hour.time.dt.hour.isin(night_subset)
)

### 3b) Sum the number of nighttime hours above a threshold of 80°F per day and per year
Like what we did above in Step 2a and 2b, we'll sum the nighttime Heat Index values for analysis. We'll use the same threshold as above, 91°F. But you can modify easily by setting this to any value of interest.

In [38]:
# sum per day
num_heatidx_hist_nighthours = (
    (heatidx_hist_nighthours >= hi_threshold).resample(time="1D").sum()
)
num_heatidx_hist_nighthours.name = (
    "Nighttime hours per day above Heat Index threshold of {}°F".format(hi_threshold)
)

In [39]:
# visualize
num_heatidx_hist_nighthours.hvplot.line(x="time", by="simulation")

:DynamicMap   [scenario]
   :NdOverlay   [simulation]
      :Curve   [time]   (Nighttime hours per day above Heat Index threshold of 91°F)

In [40]:
# sum per year
num_heatidx_hist_nighthours_per_year = num_heatidx_hist_nighthours.groupby(
    "time.year"
).sum("time")
num_heatidx_hist_nighthours_per_year.name = (
    "Nighttime hours per year above Heat Index threshold of {}°F".format(hi_threshold)
)

In [41]:
# visualize
num_heatidx_hist_nighthours_per_year.hvplot.line(x="year", by="simulation")

:DynamicMap   [scenario]
   :NdOverlay   [simulation]
      :Curve   [year]   (Nighttime hours per year above Heat Index threshold of 91°F)

We'll calculate the same steps for the projected future data, but won't visualize it here. We're just including it in case you need this information.

In [42]:
# subset for nighttime hours
heatidx_proj_nighthours = heatidx_proj_hour.isel(
    time=heatidx_proj_hour.time.dt.hour.isin(night_subset)
)

# sum per day
num_heatidx_proj_nighthours = (
    (heatidx_proj_nighthours >= hi_threshold).resample(time="1D").sum()
)
num_heatidx_proj_nighthours.name = (
    "Nighttime hours per day above Heat Index threshold of {}°F".format(hi_threshold)
)

# sum per year
num_heatidx_proj_nighthours_per_year = num_heatidx_proj_nighthours.groupby(
    "time.year"
).sum("time")
num_heatidx_proj_nighthours_per_year.name = (
    "Nighttime hours per year above Heat Index threshold of {}°F".format(hi_threshold)
)

### 3c) Export counts of nighttime hours per year

Similar the above export steps, we'll export the counts of nighttime temperatures per year next. 

In [43]:
# calculate summary stats
num_heatidx_hist_nighthours_per_year = compute_multimodel_stats(
    num_heatidx_hist_nighthours_per_year
)

# table format with summary stats
df_to_export = summary_table(num_heatidx_hist_nighthours_per_year)
df_to_export.head(5)  # see first 5 rows

Nighttime hours per year above Heat Index threshold of 91°F  \
simulation                                         WRF_CESM2_r11i1p1f1   
scenario                                        Historical + SSP 3-7.0   
year                                                                     
1980                                                      0.0            
1981                                                      1.0            
1982                                                      0.0            
1983                                                      0.0            
1984                                                      4.0            

                                                          \
simulation                                                 
scenario   Historical + SSP 2-4.5 Historical + SSP 5-8.5   
year                                                       
1980                          0.0                    0.0   
1981                          1.0                    1.0   
1982                          0.0                    0.0   
1983                          0.0                    0.0   
1984                          4.0                    4.0   

                                                            \
simulation WRF_CNRM-ESM2-1_r1i1p1f2                          
scenario     Historical + SSP 3-7.0 Historical + SSP 2-4.5   
year                                                         
1980                            0.0                    0.0   
1981                            0.0                    0.0   
1982                            1.0                    0.0   
1983                            0.0                    0.0   
1984                            0.0                    0.0   

                                                              \
simulation                        WRF_EC-Earth3-Veg_r1i1p1f1   
scenario   Historical + SSP 5-8.5     Historical + SSP 3-7.0   
year                                                           
1980                          0.0                        0.0   
1981                          0.0                        0.0   
1982                          0.0                       15.0   
1983                          0.0                        5.0   
1984                          0.0                        0.0   

                                                          \
simulation                                                 
scenario   Historical + SSP 2-4.5 Historical + SSP 5-8.5   
year                                                       
1980                          0.0                    0.0   
1981                          0.0                    0.0   
1982                          0.0                    0.0   
1983                          0.0                    0.0   
1984                          0.0                    0.0   

                                   ...                         \
simulation WRF_EC-Earth3_r1i1p1f1  ...        simulation mean   
scenario   Historical + SSP 3-7.0  ... Historical + SSP 5-8.5   
year                               ...                          
1980                          0.0  ...                  0.000   
1981                          5.0  ...                  0.125   
1982                          5.0  ...                  0.000   
1983                          3.0  ...                  0.000   
1984                          1.0  ...                  0.500   

                                                          \
simulation         simulation min                          
scenario   Historical + SSP 3-7.0 Historical + SSP 2-4.5   
year                                                       
1980                          0.0                    0.0   
1981                          0.0                    0.0   
1982                          0.0                    0.0   
1983                          0.0                    0.0   
1984                          0.0                    0.0   


In [44]:
fn = "num_heatidx_nighthours_per_year_{}".format(
    station_name.replace(" ", "_").replace("(", "").replace(")", "")
)
df_to_export.to_csv("{}.csv".format(fn))

## Step 4: Summarize the long term trends

### 4a) Comparing historical and projected number of days in each year above a Heat Index threshold

Using the [OSHA thresholds we noted above](https://www.nalc.org/workplace-issues/body/OSHA-Using-the-Heat-Index-A-Guide-for-Employers.pdf), we'll now look at a threshold of 91°F. Again, we **strongly recommend** looking at multiple thresholds to understand Heat Index trends.

In [45]:
hi_threshold = 91  # degF

If you would like to look at a specific month or season, uncomment the next cell below. We are going to look at the entire year, and leave this cell commented out. You can also skip the next cell entirely if you want to retain information throughout the entire year. 

In [46]:
# month_subset = [5, 6, 7, 8, 9] # May, June, July, August, September
# heatidx_hist_day = heatidx_hist_day.isel(time = heatidx_hist_day.time.dt.month.isin(month_subset)) # historical
# heatidx_proj_day = heatidx_proj_day.isel(time = heatidx_proj_day.time.dt.month.isin(month_subset)) # future

Let's visualize the historical trends next. 

In [47]:
ckg.view(heatidx_hist_day)

:DynamicMap   [scenario,simulation]
   :Curve   [time]   (NOAA Heat Index)

In [48]:
What about the projected trends?

Object `trends` not found.


In [49]:
ckg.view(heatidx_proj_day)

:DynamicMap   [scenario,simulation]
   :Curve   [time]   (NOAA Heat Index)

Compare the median historical and projected Heat Index values.

In [50]:
# what is the projected change in the median daily max heat index value?
hi_hist = heatidx_hist_day.median().values
hi_proj = heatidx_proj_day.median().values
print("Historical median Heat Index: {:.2f}".format(hi_hist))
print("Projected median Heat Index: {:.2f}".format(hi_proj))
print(
    "The projected change in the median Heat Index value from historical is: {:.2f}°F".format(
        hi_proj - hi_hist
    )
)

Historical median Heat Index: 73.29
Projected median Heat Index: 77.97
The projected change in the median Heat Index value from historical is: 4.67°F


### 4b) Compare the historical and projected number of days above the threshold

In [51]:
# calculate number of days above threshold
num_heatidx_histdays = (
    (heatidx_hist_day >= hi_threshold).groupby("time.year").sum("time")
)
num_heatidx_histdays.name = "Days above Heat Index threshold of {}°F".format(
    hi_threshold
)

# calculate summary statistics
num_heatidx_histdays_stats = compute_multimodel_stats(num_heatidx_histdays)

Let's visualize the trends:

In [53]:
num_heatidx_histdays.hvplot.line(x="year", by="simulation", title="") * trendline(
    num_heatidx_histdays_stats, kind="median"
).hvplot.line(x="year", color="black", line_dash="dashed", label="trendline")

ValueError: operands could not be broadcast together with shapes (3,) (11,) 

Note, if you've modified the threshold to be a higher value, for example 105°F, if the per-model trends and the median trend return flat lines, this indicates that for this location, the Heat Index value has not exceeded the modified threshold in the modeled data.

We'll now repeat the process for the projected data.

In [54]:
# calculate number of days above the threshold
num_heatidx_projdays = (
    (heatidx_proj_day >= hi_threshold).groupby("time.year").sum("time")
)
num_heatidx_projdays.name = "Days above Heat Index threshold of {}°F".format(
    hi_threshold
)

# calculate statistics
num_heatidx_projdays_stats = compute_multimodel_stats(num_heatidx_projdays)

In [55]:
# visualize results
num_heatidx_projdays.hvplot.line(x="year", by="simulation", title="") * trendline(
    num_heatidx_projdays_stats, kind="median"
).hvplot.line(x="year", color="black", line_dash="dashed", label="trendline")

:DynamicMap   [scenario]
   :Overlay
      .NdOverlay.I     :NdOverlay   [simulation]
         :Curve   [year]   (Days above Heat Index threshold of 91°F)
      .Curve.Trendline :Curve   [year]   (trendline)

### 4c) Export the number of days above a threshold

In [56]:
# table format with summary stats
df_to_export = summary_table(num_heatidx_projdays_stats)
df_to_export.head(5)  # see first 5 rows

Days above Heat Index threshold of 91°F                      \
simulation                     WRF_MIROC6_r1i1p1f1 WRF_CESM2_r11i1p1f1   
scenario                                 SSP 3-7.0           SSP 3-7.0   
year                                                                     
2039                                           0.0                 0.0   
2040                                          67.0                56.0   
2041                                          70.0                55.0   
2042                                          75.0                69.0   
2043                                          62.0                71.0   

                                                          \
simulation WRF_EC-Earth3_r1i1p1f1 WRF_FGOALS-g3_r1i1p1f1   
scenario                SSP 3-7.0              SSP 3-7.0   
year                                                       
2039                          0.0                    0.0   
2040                         97.0                   25.0   
2041                         79.0                   17.0   
2042                         76.0                   21.0   
2043                         75.0                   48.0   

                                                                \
simulation WRF_CNRM-ESM2-1_r1i1p1f2 WRF_EC-Earth3-Veg_r1i1p1f1   
scenario                  SSP 3-7.0                  SSP 3-7.0   
year                                                             
2039                            0.0                        0.0   
2040                           50.0                       58.0   
2041                           58.0                       83.0   
2042                           36.0                       94.0   
2043                           57.0                       55.0   

                                                                            \
simulation WRF_MPI-ESM1-2-HR_r3i1p1f1 WRF_TaiESM1_r1i1p1f1 simulation mean   
scenario                    SSP 3-7.0            SSP 3-7.0       SSP 3-7.0   
year                                                                         
2039                              0.0                  0.0           0.000   
2040                             79.0                 92.0          65.500   
2041                             76.0                 87.0          65.625   
2042                             84.0                118.0          71.625   
2043                             71.0                 77.0          64.500   

                                                            
simulation simulation min simulation max simulation median  
scenario        SSP 3-7.0      SSP 3-7.0         SSP 3-7.0  
year                                                        
2039                  0.0            0.0               0.0  
2040                 25.0           97.0              62.5  
2041                 17.0           87.0              73.0  
2042                 21.0          118.0              75.5  
2043                 48.0           77.0              66.5

In [57]:
fn = "num_heatidx_projdays_stats_{}".format(
    station_name.replace(" ", "_").replace("(", "").replace(")", "")
)
df_to_export.to_csv("{}.csv".format(fn))

**Note**: Any of the data variables that we've calculated throughout this notebook can be exported to .csv, like we've demonstrated throughout this notebook. Feel free to use this code to export any other variables of interest.

## Step 5: Export WRF data
In the cell below we also provide the optionality to export the input WRF data too. We demonstrate with the historical hourly Heat Index values here, but you can replace `heatidx_hist_hour` with your dataset of choice, including `heatidx_proj_hour`!

In [58]:
ck.export(heatidx_hist_hour, "my_filename", "netcdf")

Exporting specified data to NetCDF...
Saving file locally as NetCDF4...
Saved! You can find your file in the panel to the left and download to your local machine from there.
